# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [3]:
# Path to dataset
file_path_1 = Path('Data/fatal-police-shootings-data-altered.csv')
df = pd.read_csv(file_path_1)
df.head()

,age,gender,signs_of_mental_illness,threat_level,flee,body_camera,african_american
0,53.0,1.0,1,1,0.0,0,0.0
1,47.0,1.0,0,1,0.0,0,0.0
2,23.0,1.0,0,0,0.0,0,0.0
3,32.0,1.0,1,1,0.0,0,0.0
4,39.0,1.0,0,1,0.0,0,0.0


In [4]:
# Get dataset length
len(df)

6214

In [5]:
columns: [
    'age',
    'gender',
    'signs_of_mental_illness',
    'threat_level',
    'flee',
    'body_camera',
    'african_american'
]
    
target = ["african_american"]

In [6]:
# drop all rows with any NaN and NaT values
df = df.dropna()

# Split the Data into Training and Testing

In [7]:
from path import Path
import numpy as np
import pandas as pd

In [8]:
# Create our features
X = df.drop(columns="african_american")

# Create our target
y = df["african_american"]

In [9]:
X.describe()

,age,gender,signs_of_mental_illness,threat_level,flee,body_camera
count,5253.00000,5253.000000,5253.000000,5253.000000,5253.000000,5253.000000
mean,36.60080,0.952979,0.235675,0.654483,0.314106,0.134590
std,12.75049,0.211703,0.424460,0.475582,0.464203,0.341317
min,6.00000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,27.00000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,34.00000,1.000000,0.000000,1.000000,0.000000,0.000000
75%,45.00000,1.000000,0.000000,1.000000,1.000000,0.000000
max,91.00000,1.000000,1.000000,1.000000,1.000000,1.000000


In [10]:
y.describe()

count    5253.000000
mean        0.266324
std         0.442078
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000
Name: african_american, dtype: float64

In [11]:
# Split into training and testing sets

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
   y,  random_state=1, stratify=y)
X_train.shape

(3939, 6)

In [12]:
from sklearn.svm import SVC
model = SVC(kernel='linear')

In [13]:
model.fit(X_train, y_train)

SVC(kernel='linear')

In [14]:
y_pred = model.predict(X_test)
results = pd.DataFrame({
   "Prediction": y_pred,
   "Actual": y_test
}).reset_index(drop=True)
results.head()

,Prediction,Actual
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0


In [15]:
# Check accuracy

from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.7336377473363774

In [16]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.73      1.00      0.85       964
         1.0       0.00      0.00      0.00       350

    accuracy                           0.73      1314
   macro avg       0.37      0.50      0.42      1314
weighted avg       0.54      0.73      0.62      1314



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

View the count of the target classes using Counter from the collections library.
Use the resampled data to train a logistic regression model.
Calculate the balanced accuracy score from sklearn.metrics.
Print the confusion matrix from sklearn.metrics.
Generate a classication report using the imbalanced_classification_report from imbalanced-learn.
Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

## Naive Random Oversampling

In [17]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

Counter(y_resampled)

Counter({0.0: 2890, 1.0: 2890})

In [18]:
# Train the Logistic Regression model using the resampled data

from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [19]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.6052786010669828

In [20]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[539, 425],
       [122, 228]], dtype=int64)

In [21]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

        0.0       0.82      0.56      0.65      0.66      0.60      0.36       964
        1.0       0.35      0.65      0.56      0.45      0.60      0.37       350

avg / total       0.69      0.58      0.63      0.61      0.60      0.36      1314



### SMOTE Oversampling

In [22]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1,
sampling_strategy='auto').fit_resample(
   X_train, y_train)

Counter(y_resampled)

Counter({0.0: 2890, 1.0: 2890})

In [23]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)



LogisticRegression(random_state=1)

In [24]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)


0.6125489033787789

In [25]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[542, 422],
       [118, 232]], dtype=int64)

In [26]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

        0.0       0.82      0.56      0.66      0.67      0.61      0.37       964
        1.0       0.35      0.66      0.56      0.46      0.61      0.38       350

avg / total       0.70      0.59      0.64      0.61      0.61      0.37      1314



### Balanced Random Forest Classifier

In [27]:
# Resample the training data with the BalancedRandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=28)
rf_model = rf_model.fit(X_train, y_train)

In [28]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

y_pred = rf_model.predict(X_test)
acc_score = balanced_accuracy_score(y_test, y_pred)

In [29]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

# Create df for confusion matrix
cm_df = pd.DataFrame(
    cm, index=["Actual African American", "Actual Other"], columns=["Predicted African American", "Predicted Other"])

cm_df

,Predicted African American,Predicted Other
Actual African American,862,102
Actual Other,295,55


In [30]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

# Displaying results
print("Random Forest Analysis")
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report_imbalanced(y_test, y_pred))

Random Forest Analysis
Confusion Matrix


,Predicted African American,Predicted Other
Actual African American,862,102
Actual Other,295,55


Accuracy Score : 0.5256668642560759
Classification Report
                   pre       rec       spe        f1       geo       iba       sup

        0.0       0.75      0.89      0.16      0.81      0.37      0.15       964
        1.0       0.35      0.16      0.89      0.22      0.37      0.13       350

avg / total       0.64      0.70      0.35      0.65      0.37      0.15      1314



In [31]:
# List the features sorted in descending order by feature importance
importances = rf_model.feature_importances_
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.8031689441361779, 'age'),
 (0.04857180740633141, 'signs_of_mental_illness'),
 (0.044326706665310946, 'body_camera'),
 (0.04376786223623349, 'threat_level'),
 (0.036323097423397656, 'flee'),
 (0.023841582132548608, 'gender')]

### Easy Ensemble AdaBoost Classifier¶

In [32]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
ee_model = EasyEnsembleClassifier(n_estimators=100, random_state=28)
ee_model = ee_model.fit(X_train, y_train)

In [33]:
# Calculated the balanced accuracy score
y_pred = ee_model.predict(X_test)
acc_score = balanced_accuracy_score(y_test, y_pred)

In [34]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Create df for confusion matrix
cm_df = pd.DataFrame(
    cm, index=["Actual African American", "Actual Other"], columns=["Predicted African American", "Predicted Other"])

cm_df

,Predicted African American,Predicted Other
Actual African American,592,372
Actual Other,142,208


In [35]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

# Displaying results
print("Easy Ensemble Analysis")
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report_imbalanced(y_test, y_pred))

Easy Ensemble Analysis
Confusion Matrix


,Predicted African American,Predicted Other
Actual African American,592,372
Actual Other,142,208


Accuracy Score : 0.6041967990515709
Classification Report
                   pre       rec       spe        f1       geo       iba       sup

        0.0       0.81      0.61      0.59      0.70      0.60      0.37       964
        1.0       0.36      0.59      0.61      0.45      0.60      0.36       350

avg / total       0.69      0.61      0.60      0.63      0.60      0.37      1314

